In [25]:
import pandas as pd
import sqlite3
from pathlib import Path


### in UI , we have use auto select descion so all that samples comes with auto selection, but we have also REVIEW--> THAT MEANS WE HAVE TO MANUALLY LOOK THE SAMPLE FOR EX 
### SAMPLE 5 ----> REVIEW SO , WE WILL LOOK TO ALL THE ORGINAL FEATURES 
age              34
workclass        Private
education        HS-grad
education.num    9
occupation       Other-service
marital.status   Divorced
relationship     Unmarried
sex              Female
hours.per.week   45
capital.gain     0
capital.loss     3770
native.country   United-States
income           <=50K   (dataset label)

### 1️⃣ Does the data itself look valid?

Ask:

Any impossible values? ❌ No

Any missing values? ❌ No

Any contradictions? ❌ No

✔ Data is realistic and consistent

👉 So DO NOT REMOVE.

### 2️⃣ Is the dataset label reasonable?
  income <=50K

Check against human logic:

Age 34 → early/mid career

Education: HS-grad (not college)

Occupation: Other-service (typically low/moderate pay)

Work hours: 45/week (normal)

Capital gain: 0

Capital loss: 3770 (one-time event, not income)

🧠 Human reasoning:

This profile strongly matches <=50K income

✔ Label looks correct

### 3️⃣ Could the model be right instead?

If the model predicted >50K, ask:

Does this person have high-paying signals? ❌ No

Advanced education? ❌ No

High capital gains? ❌ No

👉 Model prediction >50K would be unlikely

### 4️⃣ Final human decision
### ✅ BEST ACTION: ACCEPT
Why?

Data is valid

Label makes sense

No reason to correct

No reason to remove

In [26]:
PROJECT_ROOT = Path("..").resolve()

DB_PATH = PROJECT_ROOT / "memory" / "memory.db"
SNAPSHOT_PATH = PROJECT_ROOT / "memory" / "memory_snapshot.csv"

conn = sqlite3.connect(DB_PATH)

memory_df = pd.read_sql("SELECT * FROM decisions", conn)

print("Total raw decision rows:", len(memory_df))
memory_df.head()


Total raw decision rows: 6513


,sample_index,auto_action,final_action,reason,corrected_label,risk_score,anomaly_risk,last_updated
0,0,ACCEPT,ACCEPT,Low risk,None,0.114224,0.285560,2026-01-07T22:52:58.756177
1,1,ACCEPT,ACCEPT,Low risk,None,0.075265,0.188163,2026-01-07T22:52:58.756177
2,2,ACCEPT,ACCEPT,Low risk,None,0.027079,0.067696,2026-01-07T22:52:58.756177
3,3,ACCEPT,ACCEPT,Low risk,None,0.176581,0.441452,2026-01-07T22:52:58.756177
4,4,ACCEPT,ACCEPT,Low risk,None,0.100462,0.251155,2026-01-07T22:52:58.756177


In [27]:
memory_latest = (
    memory_df
    .sort_values("last_updated")
    .groupby("sample_index", as_index=False)
    .last()
)

print("Final reviewed samples:", len(memory_latest))
memory_latest.head()


Final reviewed samples: 6513


,sample_index,auto_action,final_action,reason,corrected_label,risk_score,anomaly_risk,last_updated
0,0,ACCEPT,ACCEPT,Low risk,None,0.114224,0.285560,2026-01-07T22:52:58.756177
1,1,ACCEPT,ACCEPT,Low risk,None,0.075265,0.188163,2026-01-07T22:52:58.756177
2,2,ACCEPT,ACCEPT,Low risk,None,0.027079,0.067696,2026-01-07T22:52:58.756177
3,3,ACCEPT,ACCEPT,Low risk,None,0.176581,0.441452,2026-01-07T22:52:58.756177
4,4,ACCEPT,ACCEPT,Low risk,None,0.100462,0.251155,2026-01-07T22:52:58.756177


In [28]:
memory_df["final_action"].value_counts()


final_action
ACCEPT     5518
REVIEW      954
CORRECT      41
Name: count, dtype: int64

In [29]:
memory_latest.to_csv(SNAPSHOT_PATH, index=False)

print("✅ Final memory snapshot saved to:")
print(SNAPSHOT_PATH)


✅ Final memory snapshot saved to:
C:\Project_Final_Year\memory\memory_snapshot.csv
